In [17]:
import argparse    #python用于解析命令行参数和选项的标准模块，用于代替已经过时的optparse模块
import os          #导入操作系统接口模块
import re          #正则表达式
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable #专门为了BP算法设计的，只对输出值是标量的有用（variable是tensor的外包装，variable类型变量的data属性存储着tensor数据，grad属性存储关于该变量的导数，creator是代表该变量的创造者）
import torch.nn as nn               #主要包含了用来搭建各个层的模块（Modules），比如全连接、二维卷积、池化等；
import torch.optim as optim         #  包含了用来更新参数的优化算法，比如SGD、AdaGrad、RMSProp、 Adam等
import dataset                      #分布式的数据集合。   
from cnn_finetune import make_model #尝试对模型进行微调，以进一步提升模型性能
from tqdm import tqdm               #Tqdm 是一个快速，可扩展的Python进度条，可以在 Python 长循环中添加一个进度提示信息，用户只需要封装任意的迭代器 tqdm(iterator)。
from PIL import Image               #调用库，包含图像类
from collections import defaultdict #defaultdict的作用是在于，当字典里的key不存在但被查找时，返回的不是keyError而是一个默认值
import numpy as np                  #NumPy系统是Python的一种开源的数值计算扩展
import matplotlib.pyplot as plt     #一个绘图库，是Python中最常用的可视化工具之一，可以非常方便地创建2D图表和一些基本的3D图表
import warnings     
from torch.serialization import SourceChangeWarning
warnings.filterwarnings("ignore", category=SourceChangeWarning)   #抑制显示的特定警告

In [18]:
# 参数解释  ：argparse是是Python标准库中推荐使用的编写命令行程序的工具，是一个用来解析命令行程序的参数的模块。
# type:输入值的类型
# help:该参数的描述，用在--help中
# metavar:在--help中的参数名称
# default:默认值

# if 1:
def test_triplet():     
    parser = argparse.ArgumentParser(
        description='Face recognition using triplet loss.')
    parser.add_argument('--train-set', type=str, default='train_set', metavar='T',   #字符串类:训练集的路径      metavar:元变量：它为帮助消息中的可选参数提供了不同的名称。在add_argument（）中为metavar关键字参数提供一个值
                        help='path of train set.')
    parser.add_argument('--batch-size', type=int, default=32, metavar='N',           #整数型：batch size for training 
                        help='input batch size for training (default: 32)')
    parser.add_argument('--test-batch-size', type=int, default=4, metavar='N',        #整数型：：batch size for testing
                        help='input batch size for testing (default: 64)')
    parser.add_argument('--epochs', type=int, default=5, metavar='N',                 #整数型：训练的epochs
                        help='number of epochs to train (default: 100)')
    parser.add_argument('--lr', type=float, default=0.005, metavar='LR',              #浮点型：学习率LR
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.9, metavar='M',           #浮点型：momentum 动量，作用是尽量保持当前梯度的变化方向
                        help='SGD momentum (default: 0.9)')
    parser.add_argument('--no-cuda', action='store_true', default=False,              #没有cuda的情况
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',                    #随机数种子：通过随机种子，通过一些复杂的数学算法，你可以得到一组有规律的随机数，而随机种子就是这个随机数的初始值。随机种子相同，得到的随机数一定也相同。
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                        help='how many batches to wait before logging training status')  #打印训练状态时等待的的batch数
    parser.add_argument('--model-name', type=str, default='resnet18', metavar='M',
                        help='model name (default: resnet50)')                     
    parser.add_argument('--dropout-p', type=float, default=0.2, metavar='D',             #浮点型：dropout(在深度学习网络的训练过程中，按照一定的概率将一部分神经网络单元暂时从网络中丢弃，相当于从原始的网络中找到一个更瘦的网络)
                        help='Dropout probability (default: 0.2)')
    parser.add_argument('--check-path', type=str,                                       #字符串：模型节点路径
                        default='checkpoints', metavar='C', help='Checkpoint path')
    parser.add_argument('--is-resume', type=bool, default=True,                           #布尔型：是否从最近一次节点恢复
                        metavar='R', help='whether resume from latest checkpoint.')

    # ----------------------参数
    args = parser.parse_args(args=[])
    args.cuda = not args.no_cuda and torch.cuda.is_available()

    # ----------------------模型
    if args.is_resume:  # 从checkpoint恢复模型
        checkpoints = os.listdir(args.check_path)
        checkpoints.sort(key=lambda x: int(re.match(r'epoch_(\d+)\.pth', x).group(1)),
                         reverse=True)
        model = torch.load(os.path.join(
            args.check_path + os.path.sep + checkpoints[0]),map_location=torch.device('cpu'))
        LATEST_MODEL_ID = int(
            re.match(r'epoch_(\d+)\.pth', checkpoints[0]).group(1))
        print('[resume from model, model id: %d]' % LATEST_MODEL_ID)
    else:  # 从训练好的模型加载
        model = make_model(args.model_name,
                           pretrained=True,
                           num_classes=62,
                           dropout_p=args.dropout_p)
    
    
    if args.is_resume:  # 从训练好的模型加载
        model = make_model(args.model_name,
                           pretrained=True,
                           num_classes=62,
                           dropout_p=args.dropout_p)

    # else:  #从checkpoint恢复模型
    #     checkpoints = os.listdir(args.check_path)
    #     checkpoints.sort(key=lambda x: int(re.match(r'epoch_(\d+)\.pth', x).group(1)),
    #                      reverse=True)
    #     model = torch.load(os.path.join(
    #         args.check_path + os.path.sep + checkpoints[0]), map_location='cpu')    #这里改成了使用CPU
           
    #     LATEST_MODEL_ID = int(
    #         re.match(r'epoch_(\d+)\.pth', checkpoints[0]).group(1))
    #     print('[resume from model, model id: %d]' % LATEST_MODEL_ID)    #获得恢复的模型
    # # print('model:\n', model)

    if args.cuda:
        model.cuda()
    else:
        model.cpu()

    # ----------------------对图片数据处理: 转换成Tensor并中心归一化
    transform = transforms.Compose([
        # transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=np.array([0.485, 0.456, 0.406]),
            std=np.array([0.229, 0.224, 0.225])),
    ])

    # ----------------------加载训练数据集
    # train_set = dataset.Triplet(args.train_set,
    #                             num_cls=62,  # 62种类
    #                             num_tripets=8000,
    #                             limit=20,  # 20
    #                             transforms=transform,
    #                             train=True,
    #                             test=False)
    train_set = dataset.Hard_Triplet(args.train_set,
                                     'checkpoints/epoch_35.pth') # 先人为指定...
    train_loader = torch.utils.data.DataLoader(train_set,
                                               batch_size=args.batch_size,
                                               shuffle=True,          #用于打乱数据集，每次都会以不同的顺序返回
                                               num_workers=2)

    # ----------------------加载测试数据集
    test_set = dataset.FACE_LFW(
        args.train_set, transforms=transform, NUM_PER_CLS=20)
    test_loader = torch.utils.data.DataLoader(test_set,
                                              args.test_batch_size,
                                              shuffle=False,           
                                              num_workers=2)

    # ----------------------可视化训练数据
    def imshow(img, title=None):
        """Imshow for Tensor."""
        # (channels,imagesize,imagesize) -> (imagesize,imagesize,channels)
        img = img.numpy().transpose((1, 2, 0))  # 将Tensor中的数据格式转换用于plt显示的格式
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = std * img + mean
        img = np.clip(img, 0.0, 1.0)

        plt.imshow(img)
        if title is not None:
            plt.title(title)
        plt.pause(0.001)  # pause a bit so that plots are updated

    # for i in range(4):  # 总共迭代4个batch的数据
    #     inputs, classes = next(iter(train_loader))  # 迭代一个batch的训练数据集
    #     out = torchvision.utils.make_grid(
    #         inputs[2])  # 每个batch有4个数据，每个数据包含3张图片的数据
    #     imshow(out)
    # ---------------------------------------------

    # ----------------------训练&测试
    # 损失函数
    criterion = nn.CrossEntropyLoss()           #计算交叉熵损失(是nn.logSoftmax()和nn.NLLLoss()的整合,可以直接使用它来替换网络中的这两个操作。)
    triplet_loss = nn.TripletMarginLoss(margin=1.2, p=2)  # 三元损失：学习目标是让Positive和Anchor之间的距离 D ( a , p ) D(a,p)D(a,p) 尽可能的小，Negative和Anchor之间的距离 D ( a , n ) D(a,n)D(a,n) 尽可能的大：   优化margin

# 函数原型:CLASS torch.nn.TripletMarginLoss(margin=1.0, p=2.0, eps=1e-06, swap=False, size_average=None, reduce=None, reduction='mean')

# margin (float) – 默认为1
# p (int) – norm degree，默认为2
# swap (bool) – The distance swap is described in detail in the paper Learning shallow convolutional feature descriptors with triplet losses by V. Balntas, E. Riba et al. 默认为False
# size_average (bool) – Deprecated
# reduce (bool) – Deprecated
# reduction (string) – 指定返回各损失值(none)，批损失均值(mean)，批损失和(sum)，默认返回批损失均值(mean)



    # 优化器
    optimizer = optim.SGD(model.parameters(),            #SGD就是随机梯度下降
                          lr=args.lr,                       #学习率较小时，收敛到极值的速度较慢。学习率较大时，容易在搜索过程中发生震荡。
                          momentum=args.momentum,            #动量加速
                          weight_decay=1e-5)  #   为了有效限制模型中的自由参数数量以避免过度拟合，可以调整成本函数。权值衰减: 加入L2正则?

    def train(epoch):
        model.train()  # 网络在train模式
        total_loss = 0.0
        total_size = 0
        for batch_idx, (data, target) in enumerate(train_loader):  # 一个batch    enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中,  target是真实标签
            # if args.cuda:
            #     data[0], target[0] = data[0].cuda(), target[0].cuda()    #没有cuda
            #     data[1], target[1] = data[1].cuda(), target[1].cuda()
            #     data[2], target[2] = data[2].cuda(), target[2].cuda()
                
            data[0], target[0] = data[0].cpu(), target[0].cpu()
            data[1], target[1] = data[1].cpu(), target[1].cpu()
            data[2], target[2] = data[2].cpu(), target[2].cpu()

            data[0], target[0] = Variable(data[0]), Variable(target[0])
            data[1], target[1] = Variable(data[1]), Variable(target[1])
            data[2], target[2] = Variable(data[2]), Variable(target[2])

            optimizer.zero_grad()   #参数梯度置零:根据pytorch中的backward()函数的计算，当网络参量进行反馈时，梯度是被积累的而不是被替换掉；但是在每一个batch时毫无疑问并不需要将两个batch的梯度混合起来累积，因此这里就需要每个batch设置一遍zero_grad 了。


            # 计算特征向量
            anchor = model.forward(data[0])     #module(data)  等价于 module.forward(data),因为有python calss 中的__call__和__init__方法.  
            positive = model.forward(data[1])
            negative = model.forward(data[2])

            # 计算分类loss
            loss_cls_0 = criterion(anchor, target[0].long())             #分别计算三类的交叉熵损失
            loss_cls_1 = criterion(positive, target[1].long())
            loss_cls_2 = criterion(negative, target[2].long())
            loss_cls = loss_cls_0 + loss_cls_1 + loss_cls_2         #分类损失

            # 计算三元组loss
            loss_tri = triplet_loss.forward(anchor, positive, negative)         #计算三元组损失(anchor, positive, negative)

            # 分类loss + triplet loss: 权重如何分配?
            loss = loss_tri + loss_cls        # 总体损失定义为分类损失和三元组损失之和

            # 统计loss
            total_loss += loss.data.cpu()[0]       #累加起来
            total_size += data[0].size(0)

            loss.backward()     #反向传播计算得到每个参数的梯度值（loss.backward()）
            optimizer.step()    #通过梯度下降执行一步参数更新（optimizer.step()）

            if batch_idx % args.log_interval == 0:           #这一步是说batch恰好被等待个数整除？
                print('Train Epoch: {} [{}/{} ({:.0f}%)], Average loss: {:.4f}'.format(             #Average loss = total_loss / total_size          
                    epoch, batch_idx * len(data[0]), len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader), total_loss / total_size))

        if args.is_resume:                                     #开始回复之前的节点模型
            model_path = args.check_path + \
                os.path.sep + 'epoch_{}.pth'.format(epoch)
            if os.path.exists(model_path):
                # 如果已经存在, 重命名模型
                print('the model already exists, rename the model and save.')
                ID = (epoch + 1) + LATEST_MODEL_ID
                print('new_id: ', ID)
                model_path = args.check_path + os.path.sep + \
                    'epoch_{}.pth'.format(ID)
            torch.save(model, model_path)
        else:                                     #和上面的有区别吗？
            if epoch % 10 == 0:
                model_path = args.check_path + \
                    os.path.sep + 'epoch_{}.pth'.format(epoch)
                if os.path.exists(model_path):
                    # 如果已经存在, 重命名模型
                    print('the model already exists, rename the model and save.')
                    ID = epoch + LATEST_MODEL_ID
                    model_path = args.check_path + os.path.sep + \
                        'epoch_{}.pth'.format(ID)
                torch.save(model, model_path)
        print('model {} saved.'.format(model_path))              


#  定义测试模式
# 在train模式下，dropout网络层会按照设定的参数p设置保留激活单元的概率（保留概率=p）;batchnorm层会继续计算数据的mean和var等参数并更新。

# 在val模式下，dropout层会让所有的激活单元都通过，而batchnorm层会停止计算和更新mean和var，直接使用在训练阶段已经学出的mean和var值。

# 该模式不会影响各层的gradient计算行为，即gradient计算和存储与training模式一样，只是不进行反传（backprobagation）

# 而with torch.no_grad（）则主要是用于停止autograd模块的工作，以起到加速和节省显存的作用，具体行为就是停止gradient计算，从而节省了GPU算力和显存，但是并不会影响dropout和batchnorm层的行为。
    
    def test():  

        model.eval()  # 网络在测试模式
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            # if args.cuda:                                       #没有cuda
            #     data, target = data.cuda(), target.cuda()
                
            data, target = data.cpu(), target.cpu()
            data, target = Variable(data, volatile=True), Variable(target)

            output = model.forward(data)  # 预测

            test_loss += criterion(output, target).data.cpu()[0]   #交叉熵损失的和
            pred = output.data.max(1, keepdim=True)[1]           # keepdim（bool）– 保持输出的维度 。 当keepdim=False时，输出比输入少一个维度（就是指定的dim求范数的维度）。
                                                                 # 而keepdim=True时，输出与输入维度相同，仅仅是输出在求范数的维度上元素个数变为1。这也是为什么有时我们把参数中的dim称为缩减的维度，因为norm运算之后，此维度或者消失或者元素个数变为1。

            correct += pred.eq(target.data.view_as(pred)).cpu().sum()         #实际标签和预测结果相同的个数之和就是正确的数量

        test_loss /= len(test_loader.dataset)         #测试集的损失=交叉熵损失和/数据长度
        print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(        #分别打印出来test_loss和correct   
            test_loss, correct, len(test_loader.dataset),
            100. * correct / float(len(test_loader.dataset))))            #计算

    for epoch in range(args.epochs):           #循环
        train(epoch)
        test()
        validate(args.check_path)


In [19]:
# ----------------------验证数据集
def validate(check_path):
    if not os.path.exists(check_path):
        print('Error: invalid checkpoints path.')
        return
    print('Checkpoint path: ', check_path)

    # 加载网络
    checkpoints = os.listdir(check_path)                 #先从路径中加载之前回复的网络结构
    checkpoints.sort(key=lambda x: int(re.match(r'epoch_(\d+)\.pth', x).group(1)),
                     reverse=True)
    model_path = os.path.join(check_path + os.path.sep + checkpoints[0])
    print('model: {}'.format(model_path))

    model = torch.load(model_path,map_location=torch.device('cpu'))
    model.eval()  # 网络在求值模式

    # 数据处理方式
    transform = transforms.Compose([
        # transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=np.array([0.485, 0.456, 0.406]),
            std=np.array([0.229, 0.224, 0.225])),
    ])

    # 加载数据
    valid_set = dataset.FACE_LFW('validate_set',
                                 transforms=transform,
                                 NUM_PER_CLS=10)
    valid_loader = torch.utils.data.DataLoader(valid_set,
                                               4,             #batch-size
                                               shuffle=False,
                                               num_workers=2)
    criterion = nn.CrossEntropyLoss()     #交叉熵损失

    valid_loss = 0.0
    correct = 0
    is_cuda = torch.cuda.is_available()
    for data, target in tqdm(valid_loader):
        # if is_cuda:                                     #没有cuda直接不判断了
        #     data, target = data.cuda(), target.cuda()
        # else:
        #     data, target = data.cpu(), target.cpu()
        
        data, target = data.cpu(), target.cpu()
            
        data, target = Variable(data, volatile=True), Variable(target)

        output = model.forward(data)  # 预测

        valid_loss += criterion(output, target).data.cpu()[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    valid_loss /= float(len(valid_loader.dataset))
    print('Valid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        valid_loss, correct, len(valid_loader.dataset),
        100. * correct / float(len(valid_loader.dataset))))


In [20]:
if __name__ == '__main__':
    test_triplet()
    
    # validate('checkpoints')
    # validate('resnet_checkpoints')
    # validate_statics('checkpoints')


# https://github.com/adambielski/siamese-triplet (pytorch triplet loss)
# https://www.ddvip.com/weixin/20171218A0236200.html (pytorch显存占用分析)
# https://blog.csdn.net/qq_14845119/article/details/76083042 (车型分类博客)

[resume from model, model id: 35]


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.